In [1]:
import os
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset


class CoronarySmallDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])
        

        image = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
        image = cv2.resize(image, (256, 256))
        
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        mask = cv2.cvtColor(mask, cv2.COLOR_RGBA2RGB)
        mask = cv2.resize(mask, (256, 256))
        
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        
        return image, mask

In [2]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms


transform = transforms.Compose([
    transforms.ToTensor()
])

train_image_dir = 'images_train\input'
train_mask_dir = 'images_train\output'
val_image_dir = 'images_val\input'
val_mask_dir = 'images_val\output'
test_image_dir = 'images_test\input'
test_mask_dir = 'images_test\output'

train_dataset = CoronarySmallDataset(train_image_dir, train_mask_dir, transform=transform)
val_dataset = CoronarySmallDataset(val_image_dir, val_mask_dir, transform=transform)
test_dataset = CoronarySmallDataset(test_image_dir, test_mask_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [3]:
from medium_RGB_model import UNet


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet()
model = model.to(device)

In [4]:
import torch.optim as optim
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F


criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for images, masks in tqdm(train_loader):
            images = images.to(device)
            masks = masks.to(device)

            # print(images.size())
            # print(images)
            # print(masks.size())
            # print(masks)

            optimizer.zero_grad()
            outputs = model(images)

            # print(outputs.size())
            # print(outputs)
            
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
        
        train_loss = train_loss / len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(device)
                masks = masks.to(device)

                outputs = model(images)
                loss = criterion(outputs, masks)
                
                val_loss += loss.item() * images.size(0)
        
        val_loss = val_loss / len(val_loader.dataset)
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), 'medium_RGB_model.pth')


In [6]:
train_model(model, train_loader, val_loader, criterion, optimizer, 20)

100%|██████████| 70/70 [01:50<00:00,  1.58s/it]


Epoch 1/20, Train Loss: 0.4876, Val Loss: 0.4403


100%|██████████| 70/70 [02:00<00:00,  1.72s/it]


Epoch 2/20, Train Loss: 0.4152, Val Loss: 0.4004


100%|██████████| 70/70 [02:04<00:00,  1.78s/it]


...

Epoch 18/20, Train Loss: 0.1076, Val Loss: 0.1087


100%|██████████| 70/70 [02:03<00:00,  1.77s/it]


Epoch 19/20, Train Loss: 0.1001, Val Loss: 0.1021


100%|██████████| 70/70 [02:00<00:00,  1.71s/it]


Epoch 20/20, Train Loss: 0.0930, Val Loss: 0.0948


In [7]:
import os
import numpy as np
import cv2
import torch
from torchvision import transforms


model.load_state_dict(torch.load('medium_RGB_model.pth'))
model.eval()

def show_image(type, image_name):
    dir = f"images_test\{type}"
    print(dir)
    img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
    img = cv2.resize(img, (256, 256))
    cv2.imshow(type, img)

def predict(image_name):
    dir = 'images_test\input'
    img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
    img = cv2.resize(img, (256, 256))
    img = transforms.ToTensor()(img).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model(img)
        pred = pred.squeeze().cpu().numpy()
        print(pred)
        # pred = (pred > 0.5).astype(np.uint8)
    R, G, B = pred[0], pred[1], pred[2]
    # cv2.imshow('R', R)
    # cv2.imshow('G', G)
    # cv2.imshow('B', B)
    pred_image = np.zeros((256, 256, 3), dtype=np.uint8)
    pred_image[..., 0] = (R * 255).astype(np.uint8)
    pred_image[..., 1] = (G * 255).astype(np.uint8)
    pred_image[..., 2] = (B * 255).astype(np.uint8)
    print(pred_image)
    cv2.imshow('pred', pred_image)
    cv2.waitKey(0)
   

In [11]:
# image_name = "13c2ur549vohc0jat2dvu3xs7q1_18.png"
# image_name = "13c2ur549vohc0jat2wd23me01_25.png"
# image_name = "131aedfhs6pnf1fvtvp49h4bhdjeabmt22_31.png"
# image_name = "131aedfhs6pnf1fvtvp49juwu7plj9dv22_40.png"
show_image("input", image_name)
show_image("output", image_name)
predict(image_name)

images_test\input
images_test\output
[[[0.06489354 0.09249171 0.07752005 ... 0.07816803 0.09584146 0.16304165]
  [0.08622505 0.11786844 0.09165733 ... 0.06677479 0.06638084 0.10933329]
  [0.09612428 0.10587302 0.09507507 ... 0.09400982 0.10231416 0.12052304]
  ...
  [0.0720532  0.08142079 0.07792237 ... 0.08686867 0.08563163 0.0847557 ]
  [0.08020062 0.0878275  0.08791363 ... 0.08764994 0.14420259 0.0819165 ]
  [0.10480506 0.08055978 0.07750595 ... 0.09916172 0.07552401 0.07974146]]

 [[0.04326858 0.06595523 0.05050734 ... 0.05467865 0.08423062 0.11345358]
  [0.05581765 0.06193518 0.06063877 ... 0.04513004 0.04543136 0.08269803]
  [0.07242306 0.06785477 0.05486333 ... 0.06699282 0.07212287 0.07215668]
  ...
  [0.06422269 0.06921019 0.06434028 ... 0.07708367 0.06482835 0.05510787]
  [0.05916672 0.07351092 0.07135915 ... 0.07989731 0.10814951 0.06684975]
  [0.06550039 0.0575699  0.05946722 ... 0.07705098 0.05949239 0.05338269]]

 [[0.06968661 0.08256023 0.06908564 ... 0.04971399 0.107901

In [11]:
model.load_state_dict(torch.load('medium_RGB_model.pth'))

def dice_coefficient(y_true, y_pred):
    smooth = 1.0
    y_true_f = y_true.view(-1)
    y_pred_f = y_pred.view(-1)
    intersection = (y_true_f * y_pred_f).sum()
    return (2. * intersection + smooth) / (y_true_f.sum() + y_pred_f.sum() + smooth)

model.eval()
test_loss = 0.0
dice_scores = []

with torch.no_grad():
    for images, masks in test_loader:
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)
        
        test_loss += loss.item() * images.size(0)
        dice_scores.append(dice_coefficient(masks, outputs))

test_loss = test_loss / len(test_loader.dataset)
mean_dice = torch.mean(torch.tensor(dice_scores))

print(f'Test Loss: {test_loss:.4f}')
print(f'Mean Dice Coefficient: {mean_dice:.4f}')


Test Loss: 0.5773
Mean Dice Coefficient: 0.6904


In [ ]:
from skimage import morphology

def post_process(prediction):
    prediction_np = prediction.cpu().numpy()
    processed = morphology.remove_small_objects(prediction_np > 0.5, min_size=100)
    processed = morphology.remove_small_holes(processed, area_threshold=100)
    return torch.tensor(processed, device=device)

post_processed_predictions = [post_process(pred) for pred in outputs]


In [ ]:
from flask import Flask, request, jsonify
import io

app = Flask(__name__)
model.load_state_dict(torch.load('model.pth'))
model.eval()

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['image']
    img_bytes = file.read()
    img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    img = cv2.resize(img, (512, 512))
    img = transforms.ToTensor()(img).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model(img)
        pred = pred.squeeze().cpu().numpy()
        pred = (pred > 0.5).astype(np.uint8)
    
    _, buffer = cv2.imencode('.png', pred)
    response = io.BytesIO(buffer)
    response.seek(0)
    return response, 200, {'Content-Type': 'image/png'}

if __name__ == '__main__':
    app.run()
